# Image Classification

In [51]:
#importing all necessary libraries

import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.feature import hog
import time
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from scipy.ndimage.measurements import label
from sklearn.metrics import  accuracy_score, classification_report,f1_score

In [52]:
#loading the train data
train_dat = glob.glob('traffic/train/*.jpg')


In [53]:
#defining color histogram function for feature extraction
def color_hist(img, nbins=32):
    ch1 = np.histogram(img[:,:,0], bins=nbins, range=(0, 256))[0]#We need only the histogram, no bins edges
    ch2 = np.histogram(img[:,:,1], bins=nbins, range=(0, 256))[0]
    ch3 = np.histogram(img[:,:,2], bins=nbins, range=(0, 256))[0]
    hist = np.hstack((ch1, ch2, ch3))
    return hist

In [54]:
#extracting features of images one by one using color histogram technique
def describe(image, mask = None):
    file_features = []
    for file in image:
        image = mpimg.imread(file)
        hist = cv2.calcHist([image], [0, 1, 2], mask, [8,8,8], [0, 256, 0, 256, 0, 256])
        #cv2.normalize(hist, hist)
        file_features.append(hist.flatten())
    return file_features

In [55]:
#extracting the features for training images and scaling it using standard scaler
train_features = describe(train_dat)

train_array = np.array(train_features)

X_scaler = StandardScaler().fit(train_array)
scaled_train = X_scaler.transform(train_array)

In [56]:
#loading the train labels and appending it with their corresponding images forming a data frame
trainlabels = pd.read_csv('traffic/train.labels',
                          header=None, names =['labels'])
target = []
for x, value in np.ndenumerate(trainlabels):
    target.append(value)
y = target
train_labels =  np.array(y).reshape(len(y),1)

In [57]:
#loading the test data
test_dat = glob.glob('traffic/test/*.jpg')



In [58]:
#extracting the features for test images and scaling it using standard scaler
test_features = describe(test_dat)
test_array = np.array(test_features)
X_scaler = StandardScaler().fit(test_array)
scaled_test = X_scaler.transform(test_array)

In [43]:
testlabels = pd.read_csv('traffic/test.labels',header=None, names =['labels'])
target = []
for x, value in np.ndenumerate(testlabels):
    target.append(value)
y = target
test_labels = np.array(y).reshape(len(y),1)

In [46]:
SVMC = SVC(kernel='poly',
              C=998144.773878184,
              gamma=0.139581940130327,
              class_weight='balanced', probability=True, random_state=8)

Three different classifiers were tested using traffic small data and Extra trees Classifier was selected for the final classification based on its F1- Score.

In [32]:
#SVM
SVMC.fit(scaled_train,trainlabels)
pred = SVMC.predict(scaled_test)

print("F1 Score: ", f1_score(testlabels,pred, average='weighted'))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


F1 Score:  0.6633101055394452


In [33]:
#Extra Trees Classifier
model = ExtraTreesClassifier(n_estimators=120, max_features=20,random_state=7)
model.fit(scaled_train, trainlabels)
pred = model.predict(scaled_test)
print("F1 Score: ", f1_score(testlabels,pred, average='weighted'))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


F1 Score:  0.7012356102387276


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
#K Nearest Neighbor Classifier
knn = KNeighborsClassifier(n_neighbors=21)
knn.fit(scaled_train,trainlabels)
pred = knn.predict(scaled_test)
print("F1 Score: ", f1_score(testlabels,pred, average='weighted'))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


F1 Score:  0.6408489505377466


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [61]:
#Final classification by Extra Trees Classifier 
model = ExtraTreesClassifier(n_estimators=120, max_features=27,random_state=7)
model.fit(scaled_train, trainlabels)
pred = model.predict(scaled_test)
f= open("Prediction11.dat","w+")
for val in pred:
    f.write(str(int(val)) + "\n")
f.close()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
